# Workout Form Correction System
### Real-time pose detection and correction using MediaPipe and OpenCV

This notebook helps you perfect your workout form by:
- Detecting your pose in real-time
- Comparing it to ideal exercise forms
- Providing visual feedback with color-coded stick figures

**Color Guide:**
- 🔵 BLUE: Target reference pose
- 🟢 GREEN: Your body parts in correct position
- 🔴 RED: Body parts that need adjustment

## Step 1: Install Required Packages
Run this cell first to install dependencies

## Step 2: Import Libraries

In [45]:
import cv2
import mediapipe as mp
import numpy as np
import math
from collections import deque
import time

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## Step 3: Initialize MediaPipe Pose Detection

In [46]:
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

print("✅ MediaPipe Pose initialized!")

✅ MediaPipe Pose initialized!


## Step 4: Define Exercise Reference Poses
These are the ideal forms for each exercise

In [47]:
# Define exercise reference poses (normalized keypoint positions)
EXERCISES = {
    "dumbbell_overhead_press": {
        "name": "Dumbbell Overhead Press",
        "keyframes": [
            # Starting position - dumbbells at shoulder level
            {
                "left_shoulder": (0.35, 0.35),
                "right_shoulder": (0.65, 0.35),
                "left_elbow": (0.30, 0.50),
                "right_elbow": (0.70, 0.50),
                "left_wrist": (0.28, 0.45),
                "right_wrist": (0.72, 0.45),
                "left_hip": (0.40, 0.65),
                "right_hip": (0.60, 0.65),
            },
            # Extended position - arms overhead
            {
                "left_shoulder": (0.35, 0.35),
                "right_shoulder": (0.65, 0.35),
                "left_elbow": (0.33, 0.20),
                "right_elbow": (0.67, 0.20),
                "left_wrist": (0.35, 0.10),
                "right_wrist": (0.65, 0.10),
                "left_hip": (0.40, 0.65),
                "right_hip": (0.60, 0.65),
            }
        ]
    },
    "dumbbell_curl": {
        "name": "Dumbbell Bicep Curl",
        "keyframes": [
            # Starting position - arms extended
            {
                "left_shoulder": (0.35, 0.35),
                "right_shoulder": (0.65, 0.35),
                "left_elbow": (0.32, 0.50),
                "right_elbow": (0.68, 0.50),
                "left_wrist": (0.30, 0.65),
                "right_wrist": (0.70, 0.65),
                "left_hip": (0.40, 0.65),
                "right_hip": (0.60, 0.65),
            },
            # Curled position - weights at shoulders
            {
                "left_shoulder": (0.35, 0.35),
                "right_shoulder": (0.65, 0.35),
                "left_elbow": (0.32, 0.50),
                "right_elbow": (0.68, 0.50),
                "left_wrist": (0.30, 0.35),
                "right_wrist": (0.70, 0.35),
                "left_hip": (0.40, 0.65),
                "right_hip": (0.60, 0.65),
            }
        ]
    },
    "overhead_tricep_extension": {
        "name": "Overhead Tricep Extension",
        "keyframes": [
            # Starting position - arms overhead
            {
                "left_shoulder": (0.35, 0.30),
                "right_shoulder": (0.65, 0.30),
                "left_elbow": (0.40, 0.20),
                "right_elbow": (0.60, 0.20),
                "left_wrist": (0.50, 0.10),
                "right_wrist": (0.50, 0.10),
                "left_hip": (0.40, 0.65),
                "right_hip": (0.60, 0.65),
            },
            # Bent position - elbows bent behind head
            {
                "left_shoulder": (0.35, 0.30),
                "right_shoulder": (0.65, 0.30),
                "left_elbow": (0.40, 0.20),
                "right_elbow": (0.60, 0.20),
                "left_wrist": (0.50, 0.35),
                "right_wrist": (0.50, 0.35),
                "left_hip": (0.40, 0.65),
                "right_hip": (0.60, 0.65),
            }
        ]
    }
}

print(f"✅ Loaded {len(EXERCISES)} exercises:")
for key, data in EXERCISES.items():
    print(f"   - {data['name']}")

✅ Loaded 3 exercises:
   - Dumbbell Overhead Press
   - Dumbbell Bicep Curl
   - Overhead Tricep Extension


In [48]:
# Utility: Record your own reference pose and save to EXERCISES
def record_reference_pose(name, duration_seconds=3, sample_rate=5):
    """Record an averaged pose from the webcam and save it as a reference in EXERCISES."""
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print('❌ Could not open camera for recording')
        return None

    samples = []
    start = time.time()
    print(f'🎬 Recording reference pose for {duration_seconds} seconds...')
    while time.time() - start < duration_seconds:
        ret, frame = cap.read()
        if not ret:
            continue
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        res = pose.process(rgb)
        if res.pose_landmarks:
            norm = normalize_pose(res.pose_landmarks.landmark)
            if norm:
                samples.append(norm)
        # sleep to control sample rate
        time.sleep(1.0 / sample_rate)

    cap.release()
    if not samples:
        print('❌ No pose samples captured')
        return None

    # Average collected samples per landmark
    averaged = {}
    parts = samples[0].keys()
    for p in parts:
        xs = [s[p][0] for s in samples if p in s]
        ys = [s[p][1] for s in samples if p in s]
        if xs and ys:
            averaged[p] = (sum(xs)/len(xs), sum(ys)/len(ys))
        else:
            averaged[p] = (0.5, 0.5)

    # Create simple keyframe with a single pose (can be expanded later)
    EXERCISES[name] = {
        'name': name.replace('_', ' ').title(),
        'keyframes': [averaged]
    }
    print(f'✅ Recorded and saved reference pose: {name}')
    return averaged

In [49]:
record_reference_pose("dumbell curls", duration_seconds=5, sample_rate=5)

🎬 Recording reference pose for 5 seconds...
✅ Recorded and saved reference pose: dumbell curls
✅ Recorded and saved reference pose: dumbell curls


{'left_shoulder': (0.6012370671544757, 0.6598582863807678),
 'right_shoulder': (0.36458916110651834, 0.631060146859714),
 'left_elbow': (0.6704345558370862, 0.915907119001661),
 'right_elbow': (0.35329470038414, 0.853197808776583),
 'left_wrist': (0.699809444802148, 1.0),
 'right_wrist': (0.32899114383118494, 0.986434280872345),
 'left_hip': (0.5478090260710035, 0.9616952155317579),
 'right_hip': (0.42531327264649527, 0.949030693088259)}